In [62]:
%load_ext autoreload
%autoreload 2
from alphatoe import evals, data, game, train
import torch
from torch.nn.functional import cross_entropy
from tqdm import tqdm
from transformer_lens import HookedTransformerConfig, HookedTransformer
import json
from transformer_lens import HookedTransformer, HookedTransformerConfig
from typing import Callable, Any
import einops

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
train_data, train_labels, test_data, test_labels = data.gen_data('minimax all')

Generating all possible games...
Generated 255168 games
Generated array of moves


100%|██████████| 255168/255168 [01:18<00:00, 3264.64it/s]


torch.Size([255168, 10, 10])
Generated all data and minimax labels


In [77]:
train_data.shape

torch.Size([204134, 10])

In [65]:
cfg = HookedTransformerConfig(
    n_layers = 1,
    n_heads = 4,
    d_model = 16,
    d_head = 4,
    d_mlp = 64,
    act_fn = "relu",
    #normalization_type=None,
    normalization_type='LN',
    d_vocab=11,
    d_vocab_out=10,
    n_ctx=10,
    init_weights=True,
    device="cuda",
    seed = 1337,
)

lr = 1e-5
weight_decay = 1e-4
test_train_split = 0.7
epochs = 10_000
batch_size = 4096

In [66]:
model = HookedTransformer(cfg).to(cfg.device)

Moving model to device:  cuda


In [67]:
loss_fn = cross_entropy
optimizer =  torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-3)

In [69]:
train_losses = []
test_losses = []
# for epoch in tqdm.tqdm(range(epochs)):
for epoch in range(epochs):
    for batch in range(0, len(train_data), batch_size):
        train_logits = train_data[batch:batch+batch_size]
        train_labels = train_labels[batch:batch+batch_size]
        rearranged_train_logits = einops.rearrange(model(train_logits), "batch seq_len one_hots -> (batch seq_len) one_hots")
        rearranged_train_labels = einops.rearrange(train_labels, "batch seq_len one_hots -> (batch seq_len) one_hots")
        train_loss = loss_fn(rearranged_train_logits,rearranged_train_labels)

        train_loss.backward()

        train_losses.append(train_loss.item())
        optimizer.step()
        optimizer.zero_grad()

        with torch.inference_mode():
            test_logits = model(test_data)

            rearranged_test_logits = einops.rearrange(test_logits, "batch seq_len one_hots -> (batch seq_len) one_hots")
            rearranged_test_labels = einops.rearrange(test_labels, "batch seq_len one_hots -> (batch seq_len) one_hots")

            test_loss = loss_fn(rearranged_test_logits, rearranged_test_labels)
            test_losses.append(test_loss.item())

    print(f"Epoch {epoch} | Train Loss: {train_loss.item()} | Test Loss: {test_loss.item()}")

ValueError: Expected input batch_size (40960) to match target batch_size (0).